In [106]:
import duckdb
import pandas

In [107]:
db_path = "../../data/database/madrid.duckdb"
connection = duckdb.connect(db_path)

In [108]:
Solicitudes = connection.execute("SELECT * FROM Solicitud").fetchdf()

In [109]:
SolicitudesAdmitidas = Solicitudes.loc[(Solicitudes['curso']=='Tres Años') &
(Solicitudes['codigo_centro']==Solicitudes['codigo_centro_adm'])].copy()

In [110]:
SolicitudesAdmitidas

,estudiante,curso,prioridad,codigo_centro,nombre_centro,provincia_centro,municipio_centro,distrito_centro,nombre_distrito_centro,provincia_baremo,...,puntuacion_antiguo_alumno,puntuacion_criterio_centro,puntuacion_hermanos_matriculados,puntuacion_tutor_trabajando,puntuacion_parto_multiple,puntuacion_familia_monoparental,puntuacion_acogida,puntuacion_violencia_genero,puntuacion_total,fecha
3,3012629,Tres Años,4,28005431,CP INF-PRI REINA VICTORIA,28,79,4.0,Salamanca,28.0,...,0,0,0,0,0,0,0,0,13.0,2023-04-13
11,3012640,Tres Años,1,28001605,CPR INF-PRI-SEC LOYOLA,28,13,NaN,None,NaN,...,0,0,0,0,0,0,0,0,0.0,2023-04-13
13,3012641,Tres Años,1,28001605,CPR INF-PRI-SEC LOYOLA,28,13,NaN,None,28.0,...,4,3,0,0,0,0,0,0,19.0,2023-04-13
44,3012655,Tres Años,1,28012354,CPR INF-PRI-SEC JESUS MARIA,28,79,4.0,Salamanca,28.0,...,0,3,15,0,0,0,0,0,41.0,2023-04-13
47,3012658,Tres Años,1,28038562,CPR INF-PRI-SEC ALKOR,28,7,NaN,None,28.0,...,0,3,0,0,0,0,0,0,15.0,2023-04-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368583,3215155,Tres Años,2,28000960,CPR INF-PRI-SEC NUESTRA SEÑORA DE LOS REMEDIOS,28,7,NaN,None,NaN,...,0,0,0,0,0,0,0,0,0.0,2023-06-20
368606,3215342,Tres Años,1,28041101,CPR INF-PRI-SEC ZOLA-ROZAS,28,127,NaN,None,28.0,...,0,0,0,0,0,0,0,0,12.0,2023-06-20
368656,3215813,Tres Años,2,28063970,CP INF-PRI AGORA,28,22,NaN,None,NaN,...,0,0,0,0,0,0,0,0,0.0,2023-06-20
368666,3215835,Tres Años,1,28001848,CP INF-PRI-SEC PRINCIPE DON FELIPE,28,22,NaN,None,NaN,...,0,0,0,0,0,0,0,0,0.0,2023-06-20


In [111]:
EstudiantesMinoritarios = SolicitudesAdmitidas.loc[SolicitudesAdmitidas['puntuacion_renta_minima']==2]['estudiante']
EstudiantesMinoritarios

770       3013002
926       3013074
1021      3013118
1740      3013448
1792      3013483
           ...   
364097    3202830
364218    3202913
364261    3202942
364526    3203264
365307    3204906
Name: estudiante, Length: 797, dtype: int64

In [112]:
EstudiantesTotalesColegio = pandas.DataFrame({
    'estudiantes': SolicitudesAdmitidas.groupby
    (['codigo_centro', 'nombre_centro'])
    .size()
}).reset_index()
EstudiantesTotalesColegio

,codigo_centro,nombre_centro,estudiantes
0,28000029,CP INF-PRI-SEC SAN BLAS,30
1,28000042,CP INF-PRI EL ALAMO,43
2,28000054,CP INF-PRI ANTONIO DE NEBRIJA,11
3,28000066,CP INF-PRI CARDENAL CISNEROS,19
4,28000078,CP INF-PRI CERVANTES,18
...,...,...,...
1243,28080475,CP INF-PRI HISPANIDAD,40
1244,28080943,CPR INF-PRI-SEC EDUCREA EL VISO,49
1245,28080980,CP INF-PRI VIRGEN DE LA OLIVA,14
1246,28081561,CPR INF-PRI-SEC GSD VALDEBEBAS,60


In [113]:
EstudiantesMinoritariosColegio = pandas.DataFrame({
    'estudiantes': SolicitudesAdmitidas
    .loc[SolicitudesAdmitidas['estudiante'].isin(EstudiantesMinoritarios)].groupby
    (['codigo_centro', 'nombre_centro'])
    .size()
}).reset_index()

EstudiantesMinoritariosColegio = pandas.merge(
    EstudiantesTotalesColegio[['codigo_centro', 'nombre_centro']], 
    EstudiantesMinoritariosColegio,
    on=['codigo_centro', 'nombre_centro'],
    how='left'  
)
EstudiantesMinoritariosColegio['estudiantes'] = EstudiantesMinoritariosColegio['estudiantes'].fillna(0).astype(int)

EstudiantesMinoritariosColegio.sort_values(by='estudiantes', ascending=False,inplace = True)

EstudiantesMinoritariosColegio

,codigo_centro,nombre_centro,estudiantes
273,28009801,CPR INF-PRI-SEC SANTO ANGEL DE LA GUARDA,10
1129,28067586,CP INF-PRI ROSA MONTERO,8
180,28005933,CP INF-PRI EDUARDO ROJO,7
698,28031051,CP INF-PRI MIGUEL SERVET,7
682,28030344,CP INF-PRI MANUEL AZAÑA,6
...,...,...,...
1241,28079448,CP INF-PRI ISABEL LA CATOLICA,0
1243,28080475,CP INF-PRI HISPANIDAD,0
1244,28080943,CPR INF-PRI-SEC EDUCREA EL VISO,0
1245,28080980,CP INF-PRI VIRGEN DE LA OLIVA,0


In [114]:
Colegios = connection.execute("SELECT * FROM Colegio").fetchdf()

def municipio_colegio(codigo_colegio):
    Colegio = Colegios.loc[Colegios['codigo_colegio']==codigo_colegio
    ].iloc[0]
    return Colegio['nombre_municipio']

SolicitudesAdmitidas['municipio'] = (SolicitudesAdmitidas['codigo_centro']
                                     .apply(municipio_colegio))

In [115]:
EstudiantesTotalesMunicipio= pandas.DataFrame({
    'estudiantes': SolicitudesAdmitidas.groupby
    (['municipio'])
    .size()
}).reset_index()
EstudiantesTotalesMunicipio

,municipio,estudiantes
0,Ajalvir,30
1,Alcalá de Henares,1450
2,Alcobendas,675
3,Alcorcón,1272
4,Aldea del Fresno,24
...,...,...
130,Villanueva de la Cañada,205
131,Villanueva del Pardillo,146
132,Villarejo de Salvanés,61
133,Villaviciosa de Odón,99


In [116]:
EstudiantesMinoritariosMunicipio = pandas.DataFrame({
    'estudiantes': SolicitudesAdmitidas
    .loc[SolicitudesAdmitidas['estudiante'].isin(EstudiantesMinoritarios)].groupby
    (['municipio'])
    .size()
}).reset_index()

EstudiantesMinoritariosMunicipio = pandas.merge(
    EstudiantesTotalesMunicipio['municipio'],
    EstudiantesMinoritariosMunicipio,
    on=['municipio'],
    how='left'
)
EstudiantesMinoritariosMunicipio['estudiantes'] = EstudiantesMinoritariosMunicipio['estudiantes'].fillna(0).astype(int)

EstudiantesMinoritariosMunicipio.sort_values(by='estudiantes', ascending=False, inplace=True)

EstudiantesMinoritariosMunicipio

,municipio,estudiantes
65,Madrid,400
82,Parla,41
1,Alcalá de Henares,35
59,Leganés,31
47,Fuenlabrada,29
...,...,...
127,Villamanta,0
129,Villanueva de Perales,0
128,Villamantilla,0
133,Villaviciosa de Odón,0


In [117]:
def segregacion_desagregada(minoritarios_n, minoritarios, estudiantes_n, estudiantes):
    if minoritarios == 0:
        return 0
    return minoritarios_n/minoritarios - estudiantes_n/estudiantes

def segregacion_colegios_CAM():
    estudiantes_totales = sum(EstudiantesTotalesColegio['estudiantes'])
    estudiantes_minoritarios = sum(EstudiantesMinoritariosColegio['estudiantes'])

    S_N = 0
    for codigo_centro in EstudiantesTotalesColegio['codigo_centro']:
        minoritarios_n = EstudiantesMinoritariosColegio.loc[EstudiantesMinoritariosColegio['codigo_centro']==codigo_centro]['estudiantes'].squeeze()
        estudiantes_n = EstudiantesTotalesColegio.loc[EstudiantesTotalesColegio['codigo_centro']==codigo_centro]['estudiantes'].squeeze()
        S_N += 0.5*abs(segregacion_desagregada(minoritarios_n, estudiantes_minoritarios, estudiantes_n, estudiantes_totales))
        
    return {
        'valor_segregacion': float(S_N), 'estudiantes': estudiantes_totales, 
        'estudiantes_minoritarios': estudiantes_minoritarios
    }

SegregacionColegiosCAM = segregacion_colegios_CAM()
SegregacionColegiosCAM

{'valor_segregacion': 0.6447043896262471,
 'estudiantes': 44009,
 'estudiantes_minoritarios': 797}

In [118]:
def segregacion_colegio_CAM(codigo_centro):
    estudiantes_totales = sum(EstudiantesTotalesColegio['estudiantes'])
    estudiantes_minoritarios = sum(EstudiantesMinoritariosColegio['estudiantes'])
    minoritarios_n = EstudiantesMinoritariosColegio.loc[EstudiantesMinoritariosColegio['codigo_centro']==codigo_centro]['estudiantes'].squeeze()
    estudiantes_n = EstudiantesTotalesColegio.loc[EstudiantesTotalesColegio['codigo_centro']==codigo_centro]['estudiantes'].squeeze()
    s_n = segregacion_desagregada(minoritarios_n, estudiantes_minoritarios, 
                             estudiantes_n, estudiantes_totales)

    return {
        'valor_segregacion': float(s_n), 'estudiantes': estudiantes_n,
        'estudiantes_minoritarios': minoritarios_n
    }

datos_colegios_CAM = []

for _, colegio in EstudiantesTotalesColegio.iterrows():
    codigo_centro = colegio['codigo_centro']
    nombre_centro = colegio['nombre_centro']

    datos_colegio = segregacion_colegio_CAM(codigo_centro)

    datos_colegios_CAM.append({
        'codigo_centro': codigo_centro,
        'nombre_centro': nombre_centro,
        'valor_segregacion': datos_colegio['valor_segregacion'],
        'estudiantes': datos_colegio['estudiantes'],
        'estudiantes_minoritarios': datos_colegio['estudiantes_minoritarios']
    })


SegregacionColegioCAM = pandas.DataFrame(datos_colegios_CAM)
SegregacionColegioCAM.sort_values(by='valor_segregacion', inplace=True)
SegregacionColegioCAM

,codigo_centro,nombre_centro,valor_segregacion,estudiantes,estudiantes_minoritarios
421,28015471,CPR INF-PRI-SEC SAN AGUSTIN,-0.003499,154,0
1231,28077865,CP INF-PRI ALFREDO DI STEFANO,-0.003249,143,0
863,28038562,CPR INF-PRI-SEC ALKOR,-0.003158,139,0
1104,28065747,CPR INF-PRI-SEC COLEGIO VALDEFUENTES,-0.002999,132,0
376,28013371,CPR INF-PRI-SEC NUESTRA SEÑORA DEL RECUERDO,-0.002999,132,0
...,...,...,...,...,...
682,28030344,CP INF-PRI MANUEL AZAÑA,0.007210,14,6
180,28005933,CP INF-PRI EDUARDO ROJO,0.007533,55,7
698,28031051,CP INF-PRI MIGUEL SERVET,0.007806,43,7
1129,28067586,CP INF-PRI ROSA MONTERO,0.009129,40,8


In [119]:
def segregacion_municipios_CAM():
    estudiantes_totales = sum(EstudiantesTotalesMunicipio['estudiantes'])
    estudiantes_minoritarios = (sum
                                (EstudiantesMinoritariosMunicipio['estudiantes']))

    S_N = 0
    for municipio in EstudiantesTotalesMunicipio['municipio']:
        minoritarios_n = EstudiantesMinoritariosMunicipio.loc[EstudiantesMinoritariosMunicipio['municipio']==municipio]['estudiantes'].squeeze()
        estudiantes_n = EstudiantesTotalesMunicipio.loc[EstudiantesTotalesMunicipio['municipio']==municipio]['estudiantes'].squeeze()
        S_N += 0.5*abs(segregacion_desagregada(minoritarios_n, estudiantes_minoritarios, estudiantes_n, estudiantes_totales))

    return {
        'valor_segregacion': float(S_N), 'estudiantes': estudiantes_totales,
        'estudiantes_minoritarios': estudiantes_minoritarios
    }

SegregacionMunicipiosCAM = segregacion_municipios_CAM()
SegregacionMunicipiosCAM

{'valor_segregacion': 0.18965132402910742,
 'estudiantes': 44009,
 'estudiantes_minoritarios': 797}

In [120]:
def segregacion_municipio_CAM(municipio):
    estudiantes_totales = sum(EstudiantesTotalesMunicipio['estudiantes'])
    estudiantes_minoritarios = sum(EstudiantesMinoritariosMunicipio['estudiantes'])
    minoritarios_n = EstudiantesMinoritariosMunicipio.loc[EstudiantesMinoritariosMunicipio['municipio']==municipio]['estudiantes'].squeeze()
    estudiantes_n = EstudiantesTotalesMunicipio.loc[EstudiantesTotalesMunicipio['municipio']==municipio]['estudiantes'].squeeze()
    s_n = segregacion_desagregada(minoritarios_n, estudiantes_minoritarios,
                                  estudiantes_n, estudiantes_totales)

    return {
        'valor_segregacion': float(s_n), 'estudiantes': estudiantes_n,
        'estudiantes_minoritarios': minoritarios_n
    }

datos_municipios_CAM = []

for municipio in EstudiantesTotalesMunicipio['municipio']:

    datos_municipio = segregacion_municipio_CAM(municipio)

    datos_municipios_CAM.append({
        'municipio': municipio,
        'valor_segregacion': datos_municipio['valor_segregacion'],
        'estudiantes': datos_municipio['estudiantes'],
        'estudiantes_minoritarios': datos_municipio['estudiantes_minoritarios']
    })


SegregacionMunicipioCAM = pandas.DataFrame(datos_municipios_CAM)
SegregacionMunicipioCAM.sort_values(by='valor_segregacion', inplace=True)
SegregacionMunicipioCAM

,municipio,valor_segregacion,estudiantes,estudiantes_minoritarios
58,Las Rozas de Madrid,-0.014565,641,0
3,Alcorcón,-0.013847,1272,12
92,Rivas-Vaciamadrid,-0.012758,948,7
14,Boadilla del Monte,-0.008925,448,1
32,Colmenar Viejo,-0.008466,483,2
...,...,...,...,...
1,Alcalá de Henares,0.010967,1450,35
59,Leganés,0.013492,1118,31
8,Aranjuez,0.015169,492,21
82,Parla,0.026766,1086,41


In [124]:
def segregacion_colegios_municipio(nombre_municipio):
    EstudiantesTotalesColegioMunicipio = EstudiantesTotalesColegio[
        EstudiantesTotalesColegio['codigo_centro'].apply(municipio_colegio) == nombre_municipio
    ]
    EstudiantesMinoritariosColegioMunicipio = EstudiantesMinoritariosColegio[
        EstudiantesMinoritariosColegio['codigo_centro'].apply(municipio_colegio) == nombre_municipio
    ]
    estudiantes_totales = sum(EstudiantesTotalesColegioMunicipio['estudiantes'])
    estudiantes_minoritarios = sum(EstudiantesMinoritariosColegioMunicipio['estudiantes'])

    S_N = 0
    for codigo_centro in EstudiantesTotalesColegioMunicipio['codigo_centro']:
        minoritarios_n = EstudiantesMinoritariosColegioMunicipio.loc[EstudiantesMinoritariosColegioMunicipio['codigo_centro']==codigo_centro]['estudiantes'].squeeze()
        estudiantes_n = EstudiantesTotalesColegioMunicipio.loc[EstudiantesTotalesColegioMunicipio['codigo_centro']==codigo_centro]['estudiantes'].squeeze()
        S_N += 0.5*abs(segregacion_desagregada(minoritarios_n, estudiantes_minoritarios, estudiantes_n, estudiantes_totales))

    return {
        'valor_segregacion': float(S_N), 'estudiantes': estudiantes_totales,
        'estudiantes_minoritarios': estudiantes_minoritarios
    }

datos_colegios_municipios = []

for nombre_municipio in EstudiantesMinoritariosMunicipio.loc[EstudiantesMinoritariosMunicipio['estudiantes']>=10]['municipio']:

    datos_municipio = segregacion_colegios_municipio(nombre_municipio)

    datos_colegios_municipios.append({
        'municipio': nombre_municipio,
        'valor_segregacion': datos_municipio['valor_segregacion'],
        'estudiantes': datos_municipio['estudiantes'],
        'estudiantes_minoritarios': datos_municipio['estudiantes_minoritarios']
    })


SegregacionColegiosMunicipio = pandas.DataFrame(datos_colegios_municipios)
SegregacionColegiosMunicipio.sort_values(by='valor_segregacion', ascending=False, inplace =True)
SegregacionColegiosMunicipio

,municipio,valor_segregacion,estudiantes,estudiantes_minoritarios
10,Alcorcón,0.727201,1272,12
0,Madrid,0.652193,19220,400
2,Alcalá de Henares,0.639507,1450,35
6,Getafe,0.627334,1459,25
8,Torrejón de Ardoz,0.616749,1015,15
3,Leganés,0.562554,1118,31
4,Fuenlabrada,0.515576,1284,29
9,Alcobendas,0.499373,675,13
1,Parla,0.442528,1086,41
5,Móstoles,0.430469,1371,29


In [130]:
def segregacion_colegio_municipio(nombre_municipio):
    EstudiantesTotalesColegioMunicipio = EstudiantesTotalesColegio[
        EstudiantesTotalesColegio['codigo_centro'].apply(municipio_colegio) == nombre_municipio
        ]
    EstudiantesMinoritariosColegioMunicipio = EstudiantesMinoritariosColegio[
        EstudiantesMinoritariosColegio['codigo_centro'].apply(municipio_colegio) == nombre_municipio
        ]
    estudiantes_totales = sum(EstudiantesTotalesColegioMunicipio['estudiantes'])
    estudiantes_minoritarios = sum(EstudiantesMinoritariosColegioMunicipio['estudiantes'])
    
    datos_colegios = []

    for _, colegio in EstudiantesTotalesColegioMunicipio.iterrows():
        codigo_centro = colegio['codigo_centro']
        minoritarios_n = EstudiantesMinoritariosColegioMunicipio.loc[EstudiantesMinoritariosColegioMunicipio['codigo_centro']==codigo_centro]['estudiantes'].squeeze()
        estudiantes_n = EstudiantesTotalesColegioMunicipio.loc[EstudiantesTotalesColegioMunicipio['codigo_centro']==codigo_centro]['estudiantes'].squeeze()

        datos_colegios.append({
            'codigo_centro': codigo_centro,
            'nombre_centro': colegio['nombre_centro'],
            'valor_segregacion': segregacion_desagregada(minoritarios_n, 
                                                         estudiantes_minoritarios, estudiantes_n, estudiantes_totales),
            'estudiantes': estudiantes_n,
            'estudiantes_minoritarios': minoritarios_n
        })
    return datos_colegios

nombre_municipio = "Alcorcón"
SegregacionMunicipal = pandas.DataFrame(segregacion_colegio_municipio(nombre_municipio))
SegregacionMunicipal.sort_values(by='valor_segregacion', inplace =True)
SegregacionMunicipal 

,codigo_centro,nombre_centro,valor_segregacion,estudiantes,estudiantes_minoritarios
25,28038562,CPR INF-PRI-SEC ALKOR,-0.109277,139,0
26,28040519,CPR INF-PRI-SEC VILLALKOR,-0.078616,100,0
31,28070871,CPR INF-PRI-SEC COLEGIO JUAN PABLO II,-0.054245,69,0
32,28071802,CP INF-PRI AGUSTIN DE ARGÜELLES,-0.036950,47,0
33,28079448,CP INF-PRI ISABEL LA CATOLICA,-0.034591,44,0
22,28037521,CP INF-PRI FERNANDO DE LOS RIOS,-0.034591,44,0
18,28032389,CP INF-PRI MIGUEL HERNANDEZ,-0.033019,42,0
0,28000856,CP INF-PRI CARMEN CONDE,-0.031447,40,0
23,28037533,CP INF-PRI BLAS DE OTERO,-0.029874,38,0
2,28000947,CPR INF-PRI-SEC SANTISIMA TRINIDAD,-0.029874,38,0
